# Demo for Linear Hybrid Payoff Bandits

In [1]:
import numpy as np
import pandas as pd
from utils import get_color
from all_simulations import multi_simulation_linear
import matplotlib.pyplot as plt
from environment import HybridBandits

In [ ]:
# Define Environment Configuration

config = {}
config['seed'] = 984397                 #np.random.randint(1098321) # Uncomment the random seed generator for random instances
print('Seed:', config['seed'])
config['model_type'] = 'Linear'
config['horizon_length'] = 5000       # Number of time steps T
config['num_labels'] = 20               # Number of actions L
config['theta_dim'] = 10                 # Dimension of theta d
config['beta_dim'] = 5                  # Dimension of beta k
config['theta_norm'] = 0.8              # Max norm of theta M
config['beta_norm'] = 0.5               # Max norm of beta_i's N
config['x_norm'] = 1.0                  # Max norm of x
config['z_norm'] = 1.0                  # Max norm of z
config['subgaussian'] = 0.01             # Subgaussianity of noise
env_name = 'Testbench'                  # Name of the simulation
delta = 0.01 # High probability guarantee

In [8]:
env = HybridBandits(name=env_name, config=config)

In [15]:
# Add the algorithms to test and their hyperparameters
algo_dict = {'HyLinUCB': {'lambda': 0.1},
             'LinUCB': {'lambda': 0.1},
             'DisLinUCB': {'lambda': 0.1},
             'HyRan': {'lambda': 1.0, 'p' : 0.5}}

In [16]:
# Code to run multiple trials

num_trials = 5  # Number of trials
T = env.T

In [17]:
rewards, regrets = multi_simulation_linear(num_trials, algo_dict, env, delta, T)

regrets_dict = {k2: np.sum(regrets[i], axis=0) / num_trials for i, k2 in enumerate(algo_dict.keys())}
df = pd.DataFrame(data=regrets_dict)

In [ ]:
T_arr = np.arange(1, T+1)
_, ax = plt.subplots(1, 1, figsize=(10, 6))
for col in df.columns:
    if col == 'SupLinUCB': # SupLinUCB not plotted
        continue
    ax.plot(T_arr, df[col].cumsum(), label=col, color=get_color(col))

ax.grid()
ax.legend(fontsize=15)
ax.set_title(f'$ d_1 $ = {config['theta_dim']}, $ d_2 $ = {config['beta_dim']}, K = {config['num_labels']}', fontsize=20)
ax.set_xlabel('Time', size=20)
ax.set_ylabel('Regret', size=20)
plt.show()